In [ ]:
# Papermill Parameters
reference_in = "GRCh38"
callers_in = "sniffles,pbsv"

In [ ]:
# run at startup
library(ggplot2)
library(dplyr)
library(ggbeeswarm)

options(scipen=999)
#By setting this option to "image/svg+xml", R will attempt to generate plots in SVG format when displayed in a Jupyter Notebook environment, if possible. This format can
#be beneficial for plots as it allows for scalability and interactive features within the notebook. However, it does not work well with papermill, so we will leave it
#options(jupyter.plot_mimetypes = "image/svg+xml")

ref<-reference_in
callers<-unlist(strsplit(callers_in, ",")) # Split string to array

In [ ]:
# Print the current working directory
cat("Current working directory:\n")
cat(getwd(), "\n")

# List files in the 'ref' directory
ref_dir <- sprintf("%s/", ref)
cat("Files in the 'ref' directory:\n")
dir(ref_dir)

# SV lengths

In [ ]:
# Import and format data - slow, importing SVs from a LOT of samples
import_len_stats <- function(filename) {
    a <- read.table(filename)
    colnames(a) <- c('svtype', 'length')
    a$length <- abs(a$length)
    return(a)
}
# Create an empty list to store data frames
array_of_callers_dfs <- list()

# Loop through each variant caller to import data and store in the list
for (caller in callers) {
    filename <- sprintf("%s/%s_all_SV_lengths_by_type.txt", ref, caller)
    df <- import_len_stats(filename)
    df$caller <- caller
    array_of_callers_dfs[[caller]] <- df
}

In [ ]:
# prepare data for the binning plot (separated because it's slow)

# "cut3" is much faster than default cut, from https://stackoverflow.com/questions/21775556/getting-same-output-as-cut-using-speedier-hist-or-findinterval
cut3 <- function(x, breaks, labels) {
  out <- .bincode(x, breaks)
  attr(out, "levels") <- labels
  class(out) <- "factor"
  out
}

bin_indels <- function(SVs) {
    SVs.indel <- SVs[which(SVs$svtype=='DEL'| SVs$svtype=='INS' | SVs$svtype=='DUP'),]
    SVs.indel$bins<-cut3(SVs.indel$length, breaks=c(50,100,500,1000,5000,10000,50000,100000,500000,1000000), labels=c('50-100bp','100-500bp','500bp-1kb','1-5kb','5-10kb','10-50kb','50-100kb','100-500kb','500kb-1Mb'))
    return(SVs.indel)
}

# extract indels and bin them
all.indel <- data.frame()  # Create an empty data frame to store the combined data
c("started binning")
for (caller_name in names(array_of_callers_dfs)) {
  current_caller <- array_of_callers_dfs[[caller_name]]
  indel_name <- paste0(caller_name, ".indel")
  assigned_bins <- bin_indels(current_caller)
  assign(indel_name, assigned_bins)
  all.indel <- rbind(all.indel, assigned_bins)  # Combine binned data frames row-wise
}
c("finished binning")

all.indel$caller <- factor(all.indel$caller, levels = unique(names(array_of_callers_dfs)))  # Set 'caller' column as a factor with unique levels
all.indel.complete <- na.omit(all.indel)  # Remove rows with NA values


    # Define plotting functions

In [ ]:
# Function to plot single size distribution
# Adapted from GATK-SV
plotSizeDistribRaw <- function(dat, svtypes, n.breaks=150, k=10,
                            min.size=50, max.size=1000000,
                            autosomal=F, biallelic=F,
                            title=NULL, legend=F, lwd.cex=1, text.cex=1){

  svtypes <- data.frame('svtype'=c("DEL","DUP","CNV","INS","INV","CPX","OTH"), 'color'=c("#D43925","#2376B2","#7459B2","#D474E0","#FA931E","#71E38C","#397246"))

  #Filter/process sizes & compute range + breaks
  filter.legend <- NULL
  sizes <- log10(dat$length)
  if(length(sizes)>0){
    xlims <- range(sizes[which(!is.infinite(sizes))],na.rm=T)
    xlims[1] <- max(c(log10(min.size),xlims[1]))
    xlims[2] <- min(c(log10(max.size),xlims[2]))
    breaks <- seq(xlims[1],xlims[2],by=(xlims[2]-xlims[1])/n.breaks)
    mids <- (breaks[1:(length(breaks)-1)]+breaks[2:length(breaks)])/2
    
    #Gather size densities per class
    dens <- lapply(svtypes$svtype,function(svtype){
      vals <- sizes[which(dat$svtype==svtype)]
      h <- hist(vals[which(!is.infinite(vals) & vals>=xlims[1] & vals<=xlims[2])],plot=F,breaks=breaks)
      h$counts[1] <- h$counts[1]+length(which(!is.infinite(vals) & vals<xlims[1]))
      h$counts[length(h$counts)] <- h$counts[length(h$counts)]+length(which(!is.infinite(vals) & vals>xlims[2]))
      return(h$counts/length(vals))
    })
    names(dens) <- svtypes$svtype
    all.vals <- sizes[which(!is.infinite(sizes) & sizes>=xlims[1] & sizes<=xlims[2])]
    all.h <- hist(all.vals,plot=F,breaks=breaks)
    dens$ALL <- as.numeric(all.h$counts/length(all.vals))
    
    #Prepare plot area
    ylims <- c(0,quantile(unlist(dens),probs=0.99,na.rm=T))
    dens <- lapply(dens,function(vals){
      vals[which(vals>max(ylims))] <- max(ylims)
      return(vals)
    })
    par(bty="n",mar=c(3.5,3.5,3,0.5))
    plot(x=xlims,y=ylims,type="n",
         xaxt="n",yaxt="n",xlab="",ylab="",yaxs="i")
    
    #Add vertical gridlines
    logscale.all <- log10(as.numeric(sapply(0:8,function(i){(1:9)*10^i})))
    logscale.minor <- log10(as.numeric(sapply(0:8,function(i){c(5,10)*10^i})))
    logscale.minor.labs <- as.character(sapply(c("bp","kb","Mb"),function(suf){paste(c(1,5,10,50,100,500),suf,sep="")}))
    logscale.minor.labs <- c(logscale.minor.labs[-1],"1Gb")
    logscale.major <- log10(as.numeric(10^(0:8)))
    abline(v=logscale.all,col="gray97")
    abline(v=logscale.minor,col="gray92")
    abline(v=logscale.major,col="gray85")
    
    #Add axes, title, and Alu/SVA/L1 ticks
    axis(1,at=logscale.all,tck=-0.015,col="gray50",labels=NA)
    axis(1,at=logscale.minor,tck=-0.0225,col="gray20",labels=NA)
    axis(1,at=logscale.major,tck=-0.03,labels=NA)
    axis(1,at=logscale.minor,tick=F,cex.axis=0.8,line=-0.4,las=2,
         labels=logscale.minor.labs)
    mtext(1,text="Size",line=2.25,cex=text.cex)
    axis(2,at=axTicks(2),tck=-0.025,labels=NA)
    axis(2,at=axTicks(2),tick=F,line=-0.4,cex.axis=0.8,las=2,
         labels=paste(round(100*axTicks(2),1),"%",sep=""))
    mtext(2,text="Fraction of SV",line=2,cex=text.cex)
    sapply(1:2,function(i){
      axis(3,at=log10(c(300,6000))[i],labels=NA,tck=-0.01)
      axis(3,at=log10(c(300,6000))[i],tick=F,line=-0.9,cex.axis=0.8,
           labels=c("Alu","L1")[i],font=3)
    })
    axis(3,at=log10(c(1000,2000)),labels=NA,tck=-0.01)
    axis(3,at=mean(log10(c(1000,2000))),tick=F,line=-0.9,cex.axis=0.8,labels="SVA",font=3)
    mtext(3,line=1.5,text=title,font=2,cex=text.cex)
    
    #Add points per SV type
    sapply(1:length(dens),function(i){
      svtype <- names(dens)[i]
      if(svtype=="ALL"){
        color <- "gray15"
        lwd <- 3
      }else if (svtype=="INS" | svtype=="DEL") {
        color <- svtypes[which(svtypes$svtype==svtype),2]
        lwd <- 2
      }else {
        color <- svtypes[which(svtypes$svtype==svtype),2]
        lwd <- 0.75
      }
      #Points per individual bin
      points(x=mids,y=dens[[i]],pch=19,cex=0.25,col=color)
      lines(x=mids,y=dens[[i]],col=color,lwd=lwd.cex*lwd)
    })
    
    #Add sv type legend
    if(legend==T){
      idx.for.legend <- which(unlist(lapply(dens,function(vals){any(!is.na(vals) & !is.infinite(vals) & vals>0)})))
      legend("right",bg=NA,bty="n",pch=NA,cex=text.cex*0.7,lwd=3,
             legend=paste(rbind(svtypes, c("ALL","gray15"))$svtype[idx.for.legend], sep=""),
             col=rbind(svtypes,c("ALL","gray15"))$color[idx.for.legend])
    }
  }else{
    par(bty="n",mar=c(3.5,3.5,3,0.5))
    plot(x=c(0,1),y=c(0,1),type="n",
         xaxt="n",yaxt="n",xlab="",ylab="",yaxs="i")
    text(x=0.5,y=0.5,labels="No Data")
    mtext(3,line=1.5,text=title,font=2,cex=text.cex)
  }
  
  #Add number of SV to plot
  axis(3,at=par("usr")[2],line=-0.9,hadj=1,tick=F,
       labels=paste("n=",prettyNum(length(sizes),big.mark=","),sep=""))
  
  #Add filter labels
  if(!is.null(filter.legend)){
    legend("topright",bg=NA,bty="n",pch=NA,legend=filter.legend,cex=text.cex)
  }
}

In [ ]:
# plot SV types by BINNED length
plot_SV_types_by_binned_length_alldata <- function(all.indel.complete) {
  svtypes <- c("DEL"="#D43925","DUP"="#2376B2","CNV"="#7459B2","INS"="#D474E0","INV"="#FA931E","CPX"="#71E38C","OTH"="#397246")

  ggplot(data=all.indel.complete, aes(x=bins)) + 
    geom_bar(aes(fill=svtype), position="fill") + 
    scale_fill_manual(values=svtypes) + 
    theme(axis.text.x = element_text(angle=45, margin=margin(t = .6, unit = "cm"))) + 
    facet_wrap(~caller,ncol=1) +
    ylab("Fraction of SVs (limited to DEL, DUP, INS)") +
    xlab("SV length")
}

# Plot BINNED lengths

In [ ]:
# plot SV types by BINNED length 
options(repr.plot.width=8, repr.plot.height=10)
plot_SV_types_by_binned_length_alldata(all.indel.complete)

In [ ]:
# plot SV types by BINNED length to PDF
pdf(sprintf("SVtype_by_len_bins.%s.pdf",ref),height=8,width=10)
plot_SV_types_by_binned_length_alldata(all.indel.complete)
dev.off()

# Plot length distributions

In [ ]:
# plot SV length distributions to SCREEN
options(repr.plot.width=10, repr.plot.height=6)

for (caller_name in names(array_of_callers_dfs)) {
    current_caller <- array_of_callers_dfs[[caller_name]]
    plotSizeDistribRaw(current_caller, svtypes, legend=T, title=caller_name)
}

In [ ]:
# plot SV length distributions to PDF
for (caller_name in names(array_of_callers_dfs)) {
    current_caller <- array_of_callers_dfs[[caller_name]]
    pdf(sprintf("%s_all_size_distrib.%s.pdf",caller_name, ref),height=4,width=12)
    plotSizeDistribRaw(current_caller, svtypes, legend=T)
    dev.off()
}

# SVs per sample

In [ ]:

all_stats <- data.frame()  # Create an empty data frame to store the combined stats data  
for (caller_name in names(array_of_callers_dfs)) {
    current_caller <- array_of_callers_dfs[[caller_name]]

    # import and format data
    caller_stats<-read.table(sprintf("%s/%s_all_sample_stats_with_cov.txt", ref, caller_name),header=T,row.names=1)
    caller_stats$caller <- caller_name
    all_stats<-rbind(all_stats, caller_stats)
}

all_stats$caller<-factor(all_stats$caller, levels=unique(names(array_of_callers_dfs)))

# format data for plotting by SV type 

sv_types <- c("DEL", "INS", "DUP", "INV", "CNV", "OTH") # Define SV types
formatted_data <- list() # Empty list to store formatted data frames

for (sv_type in sv_types) {
  subset_data <- subset(all_stats, select = c("COV", sv_type, "caller"))
  colnames(subset_data) <- c("COV", "num", "caller")
  subset_data$svtype <- sv_type
  formatted_data[[sv_type]] <- subset_data
}
# Combine formatted data frames into a single data frame
final_data <- do.call(rbind, formatted_data)

all_stats_transposed <- final_data
all_stats_transposed$svtype<-factor(all_stats_transposed$svtype, levels=sv_types)
all_stats_transposed$caller<-factor(all_stats_transposed$caller, levels=callers)


In [ ]:
# Plot num SVs per sample as 'beeswarm'
plot_SVs_per_sample_beeswarm <- function(stats) {
  require(ggplot2)
  require(ggbeeswarm)

  # set coverage bins
  stats$covbin = NA
  
  # Define the conditions and corresponding bin labels
  conditions <- list(
    list(condition = is.na(stats$COV), bin_label = "NA"),
    list(condition = !is.na(stats$COV) & stats$COV < 6, bin_label = "<6x"),
    list(condition = !is.na(stats$COV) & stats$COV >= 6 & stats$COV < 7, bin_label = "6-7x"),
    list(condition = !is.na(stats$COV) & stats$COV >= 7 & stats$COV < 9, bin_label = "7-9x"),
    list(condition = !is.na(stats$COV) & stats$COV >= 9 & stats$COV < 10, bin_label = "9-10x"),
    list(condition = !is.na(stats$COV) & stats$COV >= 10, bin_label = ">10x")
    
    # Add more conditions as needed...
  )
  for (cond in conditions) {
    if (any(cond$condition)) {
      stats[cond$condition, "covbin"] <- cond$bin_label
    }
  }

  stats$caller <- factor(stats$caller, levels=callers)
  group_labels <- data.frame('caller'=levels(stats$caller))
    
  ggplot(stats, aes(x=caller, y=ALL)) + 
    geom_beeswarm(corral = "random", corral.width=0.3, method='compactswarm',
              priority='density', size=2, aes(color=covbin)) +
    scale_y_continuous(labels = scales::comma, limits = c(0, max(stats$ALL))) + 
    theme_bw() + 
    theme(text = element_text(size = 12),
          panel.grid.major.x = element_blank(), panel.grid.minor.x = element_blank(), 
          panel.border = element_blank(), 
          axis.line = element_line(color = 'black'), 
          axis.title.x = element_text(vjust=-0.5)) +
    scale_color_manual(name="Coverage",
                     values=c(">10x"="#E1000C","9-10x"="#F07386","7-9x"="#D0CDE5","6-7x"="#85A8D0","<6x"="#0A50A1", "NA"="#D3D3D3"),
                     breaks=c(">10x","9-10x","7-9x","6-7x","<6x")) +
    stat_summary(fun = median, fun.min = median, fun.max = median,
                 geom = "crossbar", width = 0.2, col='black') +
    stat_summary(fun = median, geom = "text", hjust = 0, size=10/.pt, 
                 aes(label=scales::comma(after_stat(y))), 
                 position = position_nudge(x = 0.2)) + 
    stat_summary(fun = median, fun.min = function(x){quantile(x,probs=0.25)}, fun.max = function(x){quantile(x,probs=0.75)},
                 geom = "linerange", col='black') +
  #  geom_text(aes(caller, label=caller, y=33000),data=group_labels) +
    guides(color = guide_legend(override.aes = list(size = 6))) +
    ylab("SVs per genome")
}

In [ ]:
# plot SV counts by caller to SCREEN
options(repr.plot.width=8, repr.plot.height=8)
plot_SVs_per_sample_beeswarm(all_stats)

In [ ]:
# plot SV counts by caller to PDF
pdf(sprintf("SV_counts_by_caller_beeswarm.%s.pdf",ref),height=8,width=10)
plot_SVs_per_sample_beeswarm(all_stats)

In [ ]:
# Plot num SVs per sample as violin plot (no sample-level info, probably more ok than beeswarm)
plot_SVs_per_sample_violin <- function(stats) {
    require(ggplot2)
    stats$caller <- factor(stats$caller, levels=callers)

    plot <- ggplot(stats, aes(x=caller, y=ALL)) +
        geom_violin(scale="width",fill="whitesmoke",width=0.5) +
        geom_boxplot(width=0.03, outlier.shape = NA, notch=TRUE, fill="steelblue1") +
        theme_bw() + 
        theme(text = element_text(size = 12),
            panel.grid.major.x = element_blank(), panel.grid.minor.x = element_blank(), 
            panel.border = element_blank(), 
            axis.line = element_line(color = 'black'), 
            axis.title.x = element_text(vjust=-0.5)) +
        scale_y_continuous(labels = scales::comma, limits = c(0, max(stats$ALL))) +
        stat_summary(fun = median, geom = "text", hjust = 0, size=10/.pt, 
                 aes(label=scales::comma(after_stat(y))), 
                 position = position_nudge(x = 0.27)) +
        ylab("Total number of SVs")
    print(plot)
}

In [ ]:
# plot SV counts by caller to SCREEN
options(repr.plot.width=8, repr.plot.height=8)
plot_SVs_per_sample_violin(all_stats)

In [ ]:
# plot SV counts by caller to PDF
pdf(sprintf("SV_counts_by_caller_violin.%s.pdf",ref),height=8,width=10)
plot_SVs_per_sample_violin(all_stats)
dev.off()

# Plot num SVs x coverage 

In [ ]:
cors <- all_stats %>%
  group_by(caller) %>%
  summarize(cor=cor.test(x=COV, y=ALL)$estimate, pval=format(cor.test(x=COV, y=ALL)$p.value, scientific=TRUE, digits=1))

In [ ]:
# Plot num SVs x coverage - this shows participant-level dot-plot, may not be allowed?

library(dplyr)

cors <- all_stats %>%
  group_by(caller) %>%
  summarize(cor=cor.test(x=COV, y=ALL)$estimate, 
            pval=format(cor.test(x=COV, y=ALL)$p.value, scientific=TRUE, digits=1))

cors$caller = factor(cors$caller, levels=callers)
cors$cor <- round(cors$cor, digits=2)

#ggplot(all_stats, aes(x=jitter(cov), y=jitter(ALL))) + 
ggplot(all_stats, aes(x=COV, y=ALL)) + 
    geom_point() + geom_smooth(method='lm') + 
    geom_text(data=cors, aes(x=5, y=35000, label=paste("cor=",cor,", p-value=",pval, sep='')), hjust=0) +
    theme_bw() + 
    theme(text = element_text(size = 12), strip.text = element_text(size = 14),
        panel.grid.major.x = element_blank(), panel.grid.minor.x = element_blank(), 
        axis.line = element_line(color = 'black'), 
        axis.title.x = element_text(vjust=-0.5)) +
    scale_y_continuous(labels = scales::comma) +
    xlab("coverage") + ylab("Total number of SVs") +
    facet_wrap(~caller,ncol=1)


In [ ]:
# Plot num SVs x coverage - using geom_bin2d rather than scatterplot for hiding participant-level data

plot_num_SVs_by_cov_binned <- function(all_stats) {
    require(dplyr)

    cors <- all_stats %>%
      group_by(caller) %>%
      summarize(cor=cor.test(x=COV, y=ALL)$estimate, 
                pval=format(cor.test(x=COV, y=ALL)$p.value, scientific=TRUE, digits=1))

    cors$caller = factor(cors$caller, levels=callers)
    cors$cor <- round(cors$cor, digits=2)

    ggplot(all_stats, aes(x=COV, y=ALL)) +
        geom_bin2d(bins=c(20,5)) +
        geom_smooth(method='lm',col='blue') + 
        scale_fill_gradient(low="grey90",high="black",breaks=c(50,100,150,200,250)) +
        geom_text(data=cors, aes(x=5, y=35000, label=paste("cor=",cor,", p-value=",pval, sep='')), hjust=0, vjust=0) +
        theme_bw() + 
        theme(text = element_text(size = 12), strip.text = element_text(size = 14),
            panel.grid.major.x = element_blank(), panel.grid.minor.x = element_blank(), 
            axis.line = element_line(color = 'black'), 
            axis.title.x = element_text(vjust=-0.5)) +
        scale_y_continuous(labels = scales::comma) +
        xlab("coverage") + ylab("Total number of SVs") +
        facet_wrap(~caller,ncol=1)
}

In [ ]:
# plot number of SVs by coverage to SCREEN
options(repr.plot.width=8, repr.plot.height=8)
plot_num_SVs_by_cov_binned(all_stats)

In [ ]:
# plot number of SVs by coverage to PDF
pdf(sprintf("SV_num_by_caller_and_coverage.binned.%s.pdf",ref),height=8,width=10)
plot_num_SVs_by_cov_binned(all_stats)
dev.off()

In [ ]:
plot_numSVs_by_type_x_caller <- function(stats) {
  require(ggplot2)
  require(ggbeeswarm)
  require(ggtext)
  
  svtypes <- c("DEL"="#D43925","INS"="#D474E0","DUP"="#2376B2","INV"="#FA931E","CNV"="#7459B2","OTH"="#397246")

  ggplot(stats, aes(x=svtype, y=num)) + 
#    geom_beeswarm(corral = "random", corral.width=0.3, method='compactswarm',
#              priority='density', size=2, aes(color=svtype)) +
    scale_color_manual(name="SV type",
                       values=svtypes,
                       breaks=names(svtypes)) +
    scale_fill_manual(values=svtypes) +
    geom_violin(scale="width",aes(color=svtype,fill=svtype),width=0.4) +
    geom_boxplot(width=0.1, outlier.shape = NA, notch=TRUE, fill="white") +
    scale_y_continuous(labels = scales::comma) + 
    theme_bw() + 
    theme(text = element_text(size = 12),
          axis.text.x = element_markdown(color=unname(svtypes), face="bold", angle=90, vjust = 0.5), 
          strip.text = element_text(size = 18),
          panel.grid.major.x = element_blank(), panel.grid.minor.x = element_blank(), 
          axis.title.x = element_text(vjust=-0.5), 
          legend.position = "none") +

#    stat_summary(fun = median, fun.min = median, fun.max = median,
#                 geom = "crossbar", width = 0.2, col='black') +
    stat_summary(fun = median, geom = "text", hjust = 0, size=10/.pt, 
                 aes(label=scales::comma(after_stat(y)), color=svtype), 
                 position = position_nudge(x = 0.25) ) +
#    stat_summary(fun = median, fun.min = function(x){quantile(x,probs=0.25)}, 
#                 fun.max = function(x){quantile(x,probs=0.75)}, geom = "linerange", col='black') +
    ylab("SVs per genome") +
    xlab("SV type") +
    facet_wrap(~caller,nrow=1)
}


In [ ]:
# plot number of SVs by type x caller
options(repr.plot.width=12, repr.plot.height=6)
plot_numSVs_by_type_x_caller(all_stats_transposed)

In [ ]:
# plot number of SVs by type x caller to PDF
pdf(sprintf("SV_num_by_type_and_caller.%s.pdf",ref),height=6,width=12)
plot_numSVs_by_type_x_caller(all_stats_transposed)
dev.off()